Nonetheless, Python is quite fast. You write Python for the speed of development and expressiveness. If you need speed, Guido recommends you delegate the performance critical part to another language (Cython):  
**"At some point, you end up with one little piece of your system, as a whole, where you end up spending all your time. If you write that just as a sort of simple-minded Python loop, at some point you will see that that is the bottleneck in your system. It is usually much more effective to take that one piece and replace that one function or module with a little bit of code you wrote in C or C++ rather than rewriting your entire system in a faster language, because for most of what you're doing, the speed of the language is irrelevant." --Guido van Rossum**

vectorization witn numpy and pandas, SIMD  
threads (concurrency) (race condition, tug of war) vs processes (parallel): concurrent.futures vs multiprocessing  
asyncio is in some sense serial with indirection of a coroutine. asynchronous immplemented in 3 ways: callbacks, promises, async/await  
you don't need asyncio to do asynchronous programming  
concurrency vs parallelism vs asynchronous (event driven)
C10k problem
Spark and Dask: often a functional approach, functoolz syntax translates to Dask bags
give example of how the HBase was too slow and that Python was not the bottleneck  
runtime vs space complexity  

Raymond Hettinger, Keynote on Concurrency, PyBay 2017: https://www.youtube.com/watch?v=9zinZmE3Ogk  
https://pybay.com/site_media/slides/raymond2017-keynote/threading.html